#***DataSet***

In [1]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 60.5 MB/s eta 0:00:00


In [2]:
from faker import Faker
import random
from datetime import datetime, timedelta
import csv

fake = Faker('en_IN')
num_records = 1000
data = []

# Fixed pool for phone numbers
calling_party_pool = [''.join([str(random.randint(7, 9)) + ''.join([str(random.randint(0, 9)) for _ in range(9)])]) for _ in range(50)]
called_party_pool = calling_party_pool.copy()
### called_party_pool = [''.join([str(random.randint(7, 9)) + ''.join([str(random.randint(0, 9)) for _ in range(9)])]) for _ in range(50)]

# Mapping Phone Number to a fixed IMEI and IMSI
phone_to_imei_imsi = {
    phone: {
        "IMEI": ''.join([str(random.randint(0, 9)) for _ in range(15)]),
        "IMSI": ''.join([str(random.randint(0, 9)) for _ in range(15)])
    }
    for phone in calling_party_pool
}

# Generating Sample Call Records
for _ in range(num_records):
    calling_party = random.choice(calling_party_pool)
    # Ensure calling party and called party are not the same
    called_party = calling_party
    while called_party == calling_party:
        called_party = random.choice(called_party_pool)

    imei = phone_to_imei_imsi[calling_party]["IMEI"]
    imsi = phone_to_imei_imsi[calling_party]["IMSI"]
    call_type = random.choice(['Voice In', 'Voice Out', 'SMS In', 'SMS Out'])

    call_date = fake.date_this_year()
    start_time = fake.time()
    # Call duration (random for "Voice" types; 0 for "SMS" types)
    if 'Voice' in call_type:
        duration = random.randint(0, 10800)  # 0 second to 3 hours
    else:
        duration = 0

    start_time_obj = datetime.strptime(start_time, "%H:%M:%S")
    end_time_obj = start_time_obj + timedelta(seconds=duration)
    start_time = start_time_obj.strftime("%H:%M:%S")
    end_time = end_time_obj.strftime("%H:%M:%S") if duration > 0 else "N/A"

    data.append({
        'Calling Party': calling_party,
        'Called Party': called_party,
        'IMEI': imei,
        'IMSI': imsi,
        'Call Date': call_date,
        'Call Type': call_type,
        'Start Time': start_time,
        'End Time': end_time,
        'Duration (seconds)': duration
    })

In [3]:
csv_file = 'call_records.csv'
csv_headers = ['Calling Party', 'Called Party', 'IMEI', 'IMSI', 'Call Date', 'Call Type', 'Start Time', 'End Time', 'Duration (seconds)']

with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=csv_headers)
    writer.writeheader()
    writer.writerows(data)

print(f"Data successfully exported to {csv_file}")

Data successfully exported to call_records.csv
